In [ ]:
import geopandas as gpd
import pandas as pd
import scrapbook as sb
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from get_secret_or_env_var import environ, getenv

from flowclient import connect, get_result_by_query_id, get_geography

%matplotlib inline

In [ ]:
# Default parameter values
flowapi_url = "http://localhost:9090"
aggregation_unit = "admin2"
reference_date = "2016-01-04"
previous_notebook = "run_flows.ipynb"

In [ ]:
# Connect to FlowAPI
conn = connect(
    url=flowapi_url,
    token=environ["FLOWAPI_TOKEN"],
    ssl_certificate=getenv("SSL_CERTIFICATE_FILE"),
)

In [ ]:
# Read glued data from run_flows notebook
nb = sb.read_notebook(previous_notebook)
query_ids = nb.scraps["query_ids"].data
date_periods = nb.scraps["date_periods"].data

In [ ]:
# Get cached results
cache_results = {
    key: get_result_by_query_id(connection=conn, query_id=val)
    for key, val in query_ids.items()
}

In [ ]:
# Get geography data
agg_unit_geodata = gpd.GeoDataFrame().from_features(
    get_geography(connection=conn, aggregation_unit=aggregation_unit)
)

In [ ]:
# Display title and date periods
date_periods_string = "\n".join(
    [f"- **{key}**: [{dates[0]}, {dates[1]})  " for key, dates in date_periods.items()]
)
display(
    Markdown(
        f"""
# Above-normal {aggregation_unit} flows: {reference_date}
## Date periods
{date_periods_string} 
"""
    )
)

In [ ]:
# Calculate in/outflows above normal
flowabnormal = cache_results["flowabnormal"].set_index(["pcod_from", "pcod_to"])
flownormal = cache_results["flownormal"].set_index(["pcod_from", "pcod_to"])
flows_above_normal = (flowabnormal - flownormal).dropna().reset_index()

inflow_above_normal = (
    flows_above_normal[flows_above_normal.pcod_from != flows_above_normal.pcod_to]
    .groupby("pcod_to")
    .sum()
    .reset_index()
)
outflow_above_normal = (
    flows_above_normal[flows_above_normal.pcod_from != flows_above_normal.pcod_to]
    .groupby("pcod_from")
    .sum()
    .reset_index()
)

In [ ]:
# Join with geo data

inflow_above_normal = gpd.GeoDataFrame(
    pd.merge(inflow_above_normal, agg_unit_geodata, left_on="pcod_to", right_on="pcod")
)
outflow_above_normal = gpd.GeoDataFrame(
    pd.merge(
        outflow_above_normal, agg_unit_geodata, left_on="pcod_from", right_on="pcod"
    )
)

## Inflows above normal

In [ ]:
# Plot inflows above normal
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(35, 7))
agg_unit_geodata.plot(ax=ax, color="grey")
inflows_plot = inflow_above_normal.plot(ax=ax, column="value", legend=True)
plt.axis("off")
fig.show()

In [ ]:
# Display table
inflow_above_normal[["value", "pcod"]].style.hide_index()

## Outflows above normal

In [ ]:
# Plot outflows above normal
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(35, 7))
agg_unit_geodata.plot(ax=ax, color="grey")
outflow_above_normal.plot(ax=ax, column="value", legend=True)
plt.axis("off")
fig.show()

In [ ]:
# Display table
outflow_above_normal[["value", "pcod"]].style.hide_index()